# Project 6

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

## Exploring the dataset

In [ ]:
df = pd.read_csv('dataset/RRCA_baseflow.csv')
display(df.head())
display(df.shape)

# Make the date easier to work with
df['Date'] = df['Date'] - 693963
df['Readable_Date'] = pd.to_datetime(df.Date.astype('int'), unit='D', origin=pd.Timestamp('1900-01-01'))
df['year'] = df.Readable_Date.apply(lambda x:pd.to_datetime(x).year)
df['month'] = df.Readable_Date.apply(lambda x:pd.to_datetime(x).month)
df['day'] = df.Readable_Date.apply(lambda x:pd.to_datetime(x).day)
display(df)

bySegment = df.groupby('Segment_id').count()

segmentDateMinMax = df.groupby('Segment_id')['Date'].agg(['min', 'max']).reset_index()

### Looking at scatterplots data over time, positional data, individual segments, and doing background research.

In [ ]:
plt.figure(figsize=(20, 13))
#First lets do some scatterplots
for segment_id in df['Segment_id'].unique():
  segment_data = df[df['Segment_id'] == segment_id]
  plt.scatter(segment_data['Date'], segment_data['Observed'], label=f'Segment {segment_id}')
  
plt.xlabel('Date')
plt.ylabel('Observed Baseflow')
plt.title('Date vs Baseflow')
plt.legend(loc='best')  # Change the position of the legend to upper right
plt.show()
# First let's do some scatterplots
top_two_segments = bySegment.nlargest(2, 'Date')
display(top_two_segments)

In [ ]:
# Identify the two segments with baseflows observed below 0
segmentsBelowZero = df[df['Observed'] < 0]['Segment_id'].unique().tolist()
display(segmentsBelowZero)

for segment in segmentsBelowZero:
    df[df.Segment_id == segment].plot(x='Date', y='Observed')

# Remove these segments because it seems like there was a data collection error
for segment in segmentsBelowZero:
    df = df[df.Segment_id != segment]
df.Segment_id.unique()

In [ ]:
for segment_id in df['Segment_id'].unique():
    segment_data = df[df['Segment_id'] == segment_id]
    if segment_data['Observed'].max() > 200:
        plt.scatter(segment_data['Date'], segment_data['Observed'], label=f'Segment {segment_id}')

plt.xlabel('Date')
plt.ylabel('Observed Baseflow')
plt.title('Date vs Baseflow')
plt.legend(loc='best')  # Change the position of the legend to upper right
plt.show()

In [ ]:
# Remove segments that are outliers to the normal range of observed baseflow
segmentObservedMinMax = df.groupby('Segment_id')['Observed'].agg(['min', 'max', 'mean']).reset_index()
display(segmentObservedMinMax.sort_values('max'))
display(segmentObservedMinMax.sort_values('mean'))
outlierSegments = segmentObservedMinMax.sort_values('mean', ascending=False).head(2).Segment_id.tolist()
for outlier in outlierSegments:
    df = df[df.Segment_id != outlier]

In [ ]:
plt.figure(figsize=(20, 13))
#First lets do some scatterplots
for segment_id in df['Segment_id'].unique():
  segment_data = df[df['Segment_id'] == segment_id]
  plt.scatter(segment_data['Date'], segment_data['Observed'], label=f'Segment {segment_id}')
  
plt.xlabel('Date')
plt.ylabel('Observed Baseflow')
plt.title('Date vs Baseflow')
plt.legend(loc='best')  # Change the position of the legend to upper right
plt.show()
# First let's do some scatterplots
top_two_segments = bySegment.nlargest(2, 'Date')
display(top_two_segments)

In [ ]:
plt.scatter(df['Evapotranspiration'], df['Observed'])
plt.xlabel('Evapotranspiration')
plt.ylabel('Observed')
plt.title('Observed vs Evapotranspiration')
plt.show()

plt.scatter(df['Precipitation'], df['Observed'])
plt.xlabel('Precipitation')
plt.ylabel('Observed')
plt.title('Observed vs Precipitation')
plt.show()

plt.scatter(df['Irrigation_pumping'], df['Observed'])
plt.xlabel('Irrigation_pumping')
plt.ylabel('Observed')
plt.title('Observed vs Irrigation_pumping')
plt.show()




### Initial prediction testing

In [ ]:
feature_cols = [	'x',	'y',	'Evapotranspiration',	'Precipitation',	'Irrigation_pumping']
X = df[feature_cols]
y = df["Observed"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#Segment_id	x	y	Evapotranspiration	Precipitation	Irrigation_pumping	Observed
lm = LinearRegression()
lm.fit(X_train, y_train)


predictions = lm.predict(X_test)
print(predictions)
r_squared = r2_score(y_test, predictions)
print(r_squared)
display(X_test)

plt.scatter(range(len(predictions)), predictions, label='Predictions')
plt.xlabel('Index')
plt.ylabel('Predicted Value')
plt.legend()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Plot the actual data points
plt.scatter(X_test["Evapotranspiration"], y_test, color='blue', label='Actual Data')

# Plot the linear regression line
plt.plot(X_test["Evapotranspiration"], predictions, color='red', linewidth=2, label='Linear Regression Line')

# Add labels and title
plt.xlabel('x')
plt.ylabel('Target Variable')
plt.title('Linear Regression Model')

# Add legend
plt.legend()

# Show plot
plt.show()


### Visualizing the data

In [ ]:
sns.lineplot(data=df, x='year', y='Observed')
plt.xlabel('Year')
plt.ylabel('Observed Baseflow')
plt.title('Average Baseflow per Year of All Segments')

In [ ]:
sns.lineplot(data=df, x='month', y='Observed')
plt.xlabel('Month')
plt.ylabel('Observed Baseflow')
plt.title('Average Baseflow per Month of All Segments')

In [ ]:
sns.lineplot(data=df, x='year', y='Observed', hue="month")
plt.xlabel('Year')
plt.ylabel('Observed Baseflow')
plt.title('Average Monthly Baseflow per Year of All Segments')

In [ ]:
sns.lineplot(data=df, x='month', y='Observed', hue="year")
plt.xlabel('Month')
plt.ylabel('Observed Baseflow')
plt.title('Average Baseflow per Month by Year of All Segments')

In [ ]:
avgObservedPerYear = df.groupby('year')['Observed'].mean().reset_index()
display(avgObservedPerYear)

sns.lineplot(data=avgObservedPerYear, x='year', y='Observed')
plt.xlabel('Year')
plt.ylabel('Observed Baseflow')
plt.title('Average Baseflow per Year of All Segments')

In [ ]:
avgObservedPerMonth = df.groupby('month')['Observed'].mean().reset_index()
display(avgObservedPerMonth)

sns.lineplot(data=avgObservedPerMonth, x='month', y='Observed')
plt.xlabel('Month')
plt.ylabel('Observed Baseflow')
plt.title('Average Baseflow per Year of All Segments')

In [ ]:
df['monthsSince1900'] = df['month'] + df['year'] * 12 - 22800
display(df)

sns.lineplot(data=df, x='monthsSince1900', y='Observed')
plt.ylabel('Observed Baseflow')
plt.title('Average Baseflow per Month of All Segments')
plt.xlabel('Months since 1900')

In [ ]:
# visualize the relationship between the features and the response using scatterplots
fig, axs = plt.subplots(1, 3, sharey=True)
df.plot(kind='scatter', x='Date', y='Observed', ax=axs[0], figsize=(20, 8))
df.plot(kind='scatter', x='year', y='Observed', ax=axs[1])
df.plot(kind='scatter', x='month', y='Observed', ax=axs[2])
axs[0].set_title('Date vs Baseflow')
axs[1].set_title('Year vs Baseflow')
axs[2].set_title('Month vs Baseflow')

In [ ]:
# visualize the relationship between the features and the response using scatterplots
fig, axs = plt.subplots(1, 3, sharey=True)
df.plot(kind='scatter', x='Evapotranspiration', y='Observed', ax=axs[0], figsize=(20, 8))
df.plot(kind='scatter', x='Precipitation', y='Observed', ax=axs[1])
df.plot(kind='scatter', x='Irrigation_pumping', y='Observed', ax=axs[2])
axs[0].set_title('Evapotranspiration vs Baseflow')
axs[1].set_title('Precipitation vs Baseflow')
axs[2].set_title('Irrigation Pumping vs Baseflow')

In [ ]:
# visualize the relationship between the features and the response using scatterplots
fig, axs = plt.subplots(1, 2, sharey=True)
df.plot(kind='scatter', x='x', y='Observed', ax=axs[0], figsize=(16, 8))
df.plot(kind='scatter', x='y', y='Observed', ax=axs[1])
axs[0].set_title('X Coordinate of Segment vs Baseflow')
axs[1].set_title('Y Coordinate of Segment vs Baseflow')


In [ ]:
plt.figure(figsize=(10, 8))
sns.scatterplot(data=df, x='x', y='y', size='Observed', sizes=(50, 500), hue='year')
plt.title('X,Y Coordinates of Segments and Their Corresponding Baseflow')

In [ ]:
plt.figure(figsize=(10, 8))
sns.scatterplot(data=df, x='x', y='y', size='Precipitation', sizes=(50, 500), hue='year')
plt.title('X,Y Coordinates of Segments and Their Corresponding Precipitation')

## Linear regression models

### Split the data into two groups

In [ ]:
decreasingMonthsDF = df[df.month.isin(range(2, 8))].reset_index()
increasingMonthsDF = df[df.month.isin([1, 8, 9, 10, 11, 12])].reset_index()
increasingMonthsDF.loc[increasingMonthsDF['month'] == 1, 'month'] = 13

In [ ]:
sns.regplot(data=decreasingMonthsDF, x='month', y='Observed')
plt.title('Month vs Baseflow')
plt.xlabel('Month')
plt.ylabel('Observed Baseflow')

In [ ]:
sns.regplot(data=increasingMonthsDF, x='month', y='Observed')
plt.title('Month vs Baseflow')
plt.xlabel('Month')
plt.ylabel('Observed Baseflow')

### Linear regression testing

In [ ]:
from scipy.stats import pearsonr

# Compute the Pearson correlation coefficient and p-value
pearson_corr, p_value = pearsonr(df['month'], df['Observed'])

print("Pearson correlation coefficient:", pearson_corr)
print("p-value:", p_value)

# Compute the Pearson correlation coefficient and p-value
pearson_corr, p_value = pearsonr(decreasingMonthsDF['month'], decreasingMonthsDF['Observed'])

print("Pearson correlation coefficient:", pearson_corr)
print("p-value:", p_value)

# Compute the Pearson correlation coefficient and p-value
pearson_corr, p_value = pearsonr(increasingMonthsDF['month'], increasingMonthsDF['Observed'])

print("Pearson correlation coefficient:", pearson_corr)
print("p-value:", p_value)

In [ ]:
# create X and y
# feature_cols = ['month', 'Date', 'Evapotranspiration', 'Precipitation', 'x', 'y']
feature_cols = ['month']
X = decreasingMonthsDF[feature_cols]
y = decreasingMonthsDF.Observed

# follow the usual sklearn pattern: import, instantiate, fit
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X, y)

# print intercept and coefficients
print(lm.intercept_)
print(lm.coef_)
print(lm.score(X, y))

### Linear regression on unsplit data

In [ ]:
feature_cols = feature_cols = ['y', 'x', 'Precipitation', 'Evapotranspiration', 'month', 'Irrigation_pumping', 'Date']
crossValidations = 20
avgRSquared = 0
for _ in range(crossValidations):
        X = df[feature_cols]
        y = df['Observed']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)#, random_state=42)

        lm = LinearRegression()
        lm.fit(X_train, y_train)
        
        avgRSquared += lm.score(X_test, y_test)
avgRSquared /= crossValidations
avgRSquared

In [ ]:
import statsmodels.formula.api as smf

lm = smf.ols(formula='Observed ~ y + x + Precipitation + Evapotranspiration + month + Irrigation_pumping + Date', data=df).fit()

# print the coefficients
display(lm.params)
display(lm.pvalues)

# print a summary of the fitted model
lm.summary()

In [ ]:
# create X and y
feature_cols = ['month']
X = df[feature_cols]
y = df.Observed

# follow the usual sklearn pattern: import, instantiate, fit
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X, y)

# print intercept and coefficients
print(lm.intercept_)
print(lm.coef_)
print(lm.score(X, y))

In [ ]:
import statsmodels.formula.api as smf

lm = smf.ols(formula='Observed ~ month', data=df).fit()

# print the coefficients
display(lm.params)
display(lm.pvalues)

# print a summary of the fitted model
lm.summary()

### Linear regression on split data

#### Months 2-7

In [ ]:
feature_cols = feature_cols = ['y', 'x', 'Precipitation', 'Evapotranspiration', 'month', 'Irrigation_pumping', 'Date']
crossValidations = 20
avgRSquared = 0
for _ in range(crossValidations):
        X = decreasingMonthsDF[feature_cols]
        y = decreasingMonthsDF['Observed']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)#, random_state=42)

        lm = LinearRegression()
        lm.fit(X_train, y_train)
        
        avgRSquared += lm.score(X_test, y_test)
avgRSquared /= crossValidations
avgRSquared

In [ ]:
import statsmodels.formula.api as smf

lm = smf.ols(formula='Observed ~ y + x + Precipitation + Evapotranspiration + month + Irrigation_pumping + Date', data=decreasingMonthsDF).fit()

# print the coefficients
display(lm.params)
display(lm.pvalues)

# print a summary of the fitted model
lm.summary()

In [ ]:
# create X and y
feature_cols = ['month']
X = decreasingMonthsDF[feature_cols]
y = decreasingMonthsDF.Observed

# follow the usual sklearn pattern: import, instantiate, fit
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X, y)

# print intercept and coefficients
print(lm.intercept_)
print(lm.coef_)
print(lm.score(X, y))

In [ ]:
import statsmodels.formula.api as smf

lm = smf.ols(formula='Observed ~ month', data=decreasingMonthsDF).fit()

# print the coefficients
display(lm.params)
display(lm.pvalues)

# print a summary of the fitted model
lm.summary()

#### Months 8-12, 1

In [ ]:
feature_cols = feature_cols = ['y', 'x', 'Precipitation', 'Evapotranspiration', 'month', 'Irrigation_pumping', 'Date']
crossValidations = 20
avgRSquared = 0
for _ in range(crossValidations):
        X = increasingMonthsDF[feature_cols]
        y = increasingMonthsDF['Observed']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)#, random_state=42)

        lm = LinearRegression()
        lm.fit(X_train, y_train)
        
        avgRSquared += lm.score(X_test, y_test)
avgRSquared /= crossValidations
avgRSquared

In [ ]:
import statsmodels.formula.api as smf

lm = smf.ols(formula='Observed ~ y + x + Precipitation + Evapotranspiration + month + Irrigation_pumping + Date', data=increasingMonthsDF).fit()

# print the coefficients
display(lm.params)
display(lm.pvalues)

# print a summary of the fitted model
lm.summary()

In [ ]:
# create X and y
feature_cols = ['month']
X = increasingMonthsDF[feature_cols]
y = increasingMonthsDF.Observed

# follow the usual sklearn pattern: import, instantiate, fit
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X, y)

# print intercept and coefficients
print(lm.intercept_)
print(lm.coef_)
print(lm.score(X, y))

In [ ]:
import statsmodels.formula.api as smf

lm = smf.ols(formula='Observed ~ month', data=increasingMonthsDF).fit()

# print the coefficients
display(lm.params)
display(lm.pvalues)

# print a summary of the fitted model
lm.summary()